# Writing Custom Loss Functions

In this tutorial we will learn how to write our own loss functions and add them to EncoderMap. Let us start with the imports:

In [ ]:
import numpy as np
import encodermap as em
import pandas as pd
import tensorflow as tf

## Adding a unit circle loss

To show how to implement loss functions we will replace EncoderMap's center_cost with a loss that tries to push the low-dimensional points into a unit circle. For a unit circle the following equation holds true:

\begin{align}
x^2 + y^2 &= 1\\
x^2 + y^2 - 1 &= 0
\end{align}

Let us first plot a unit circle with matplotlib.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

t = np.linspace(0,np.pi*2,100)

plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect='equal')
ax.plot(np.cos(t), np.sin(t), linewidth=1)

**How to put this information into a loss function?**

We need to find a function that describes the distance between any (x, y)-coordinate to the unit circle.

In [ ]:
def distance_to_unit_circle_2D(x, y):
    return np.abs((np.square(x) + np.square(y)) - 1)

def distance_to_unit_circle(points):
    return np.abs(np.sum(np.square(points), axis=0) - 1)

In [ ]:
xx = np.linspace(-2, 2, 250)
yy = np.linspace(-2, 2, 250)
grid = np.meshgrid(xx, yy)
z = distance_to_unit_circle(grid)

plt.close('all')
plt.contourf(xx, yy, z, levels=60)

### Build a loss function from that:

Cost functions in EncoderMap are almost always closures. Meaning they return a function and not a value. Let's look at an example closure:

In [ ]:
def print_msg(msg):
    # This is the outer enclosing function

    def printer():
        # This is the nested function
        print(msg)

    printer()

# We execute the function
# Output: Hello
print_msg("Hello")

The printer function was able to access the non-local variable `msg`. EncoderMap's lossfunctions use the non-local variables `model` and `parameters` (often abbreviated to `p`).

We will also add `tf.reduce_mean()` to get the mean distance from the unit circle for all points, because a loss is always a scalar value.

In [ ]:
def circle_loss(model, parameters):
    """Circle loss outer function. Takes model and parameters. Parameters is only here for demonstration purpoes.
    It is not actually needed in the closure.
    
    """
    
    # use the models encoder part to create low-dimensional data
    latent = model.encoder
    
    def circle_loss_fn(y_true, y_pred=None):
        """Circle loss inner function. Takes y_true and y_pred. y_pred will not be used. y_true will be used to get
        the latent space of the autoencoder.
        
        """
        # get latent output
        lowd = latent(y_true)
        
        # get circle cost
        circle_cost = tf.reduce_mean(tf.abs(tf.reduce_sum(tf.square(lowd), axis=0) - 1))
        
        # bump up the cost
        circle_cost *= 5
        
        # write to tensorboard
        tf.summary.scalar('Circle Cost', circle_cost)
        
        # return circle cost
        return circle_cost
    
    # return inner function
    return circle_loss_fn

### Include the loss function in EncoderMap

**First:** Let us load the dihedral data from ../notebooks_easy and define some Parameters. For the parameters we will set the center_cost_scale to be 0 as to not interfere with our new circle cost.

In [ ]:
df = pd.read_csv('../notebooks_easy/asp7.csv')
dihedrals = df.iloc[:,:-1].values.astype(np.float32)
cluster_ids = df.iloc[:,-1].values
print(dihedrals.shape, cluster_ids.shape)
print(df.shape)

In [ ]:
parameters = em.Parameters(
tensorboard=True,
center_cost_scale=0,
periodicity=2*np.pi,
main_path=em.misc.run_path('runs/custom_losses')
)

Now we can instaniate the `EncoderMap` class. For visualization purposes we will also make tensorboard write images.

In [ ]:
e_map = em.EncoderMap(parameters, dihedrals)
e_map.add_images_to_tensorboard(dihedrals, image_step=1)

The loss is created by giving it the model and parameters of the parent `EncoderMap` instance. To not clash with the names of function and result we will call it `_circle_loss`.

In [ ]:
_circle_loss = circle_loss(e_map.model, e_map.p)
print(_circle_loss)

Now we add this loss to `EncoderMap`'s losses

In [ ]:
print(e_map.loss)
e_map.loss.append(_circle_loss)
print(e_map.loss)

### Train

In [ ]:
e_map.train()

### Output

Here's what Tensorboard should put out:

<img src="custom_loss_functions_1.png" width="800">